In [1]:
import cv2
print(cv2.__version__)

4.5.4-dev


In [2]:
import os
import numpy as np
import cv2
from glob import glob
def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[19:]
    

  
  
    print(name)
    print(subname)
    if (name.find("bhuj") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    else:
        save_path = os.path.join(save_dir, "Tadasana",subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            cv2.imwrite(f"{save_path}/{idx}.png", frame)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("../Yoga_Vid_Collected/*")
    save_dir = "save"

    for path in video_paths:
        save_frame(path, save_dir, gap=30)

yoga_vid_collected\abhay_bhuj
abhay_bhuj
save\Bhujangasana\abhay_bhuj
yoga_vid_collected\abhay_padam
abhay_padam
save\Padamasana\abhay_padam
yoga_vid_collected\abhay_shav
abhay_shav
save\Shavasana\abhay_shav
yoga_vid_collected\abhay_tada
abhay_tada
save\Tadasana\abhay_tada
yoga_vid_collected\abhay_trik
abhay_trik
save\Trikonasana\abhay_trik
yoga_vid_collected\abhay_vriksh
abhay_vriksh
save\Vrikshasana\abhay_vriksh
yoga_vid_collected\ameya_bhuj
ameya_bhuj
save\Bhujangasana\ameya_bhuj
yoga_vid_collected\ameya_padam
ameya_padam
save\Padamasana\ameya_padam
yoga_vid_collected\ameya_shav
ameya_shav
save\Shavasana\ameya_shav
yoga_vid_collected\ameya_tada
ameya_tada
save\Tadasana\ameya_tada
yoga_vid_collected\ameya_trik
ameya_trik
save\Trikonasana\ameya_trik
yoga_vid_collected\ameya_vriksh
ameya_vriksh
save\Vrikshasana\ameya_vriksh
yoga_vid_collected\bhumi_bhuj
bhumi_bhuj
save\Bhujangasana\bhumi_bhuj
yoga_vid_collected\bhumi_padam
bhumi_padam
save\Padamasana\bhumi_padam
yoga_vid_collected\bhum